In [1]:
import _compare_likelihoods
import lda
from scipy import sparse
import numpy as np

import json
import gensim 
import sklearn
import sys

unable to import 'smart_open.gcs', disabling that module


In [8]:
import utils
from tqdm import tqdm

### Get previous ccLDA params 

In [4]:
ccLDA_nzw = np.load("ccLDA_nzw.npy")
ccLDA_nx = np.load("ccLDA_nx.npy")
ccLDA_nz = np.load("ccLDA_nz.npy")
ccLDA_nzwc = np.load("ccLDA_nzwc.npy")
ccLDA_nzc = np.load("ccLDA_nzc.npy")

### Get data

In [3]:
X_val = sparse.load_npz('/Users/tingchen/Desktop/Yelp Project/data_n_20_removed_all_eths_less_than_15000/X_val.npz')
cc_val= np.load("/Users/tingchen/Desktop/Yelp Project/data_n_20_removed_all_eths_less_than_15000/y_val.npy")
locs_val = []
with open("/Users/tingchen/Desktop/Yelp Project/data_n_20_removed_all_eths_less_than_15000/locs_val.json", 'r') as file:
    locs_val = json.load(file)
    
X_test = sparse.load_npz('/Users/tingchen/Desktop/Yelp Project/data_n_20_removed_all_eths_less_than_15000/X_test.npz')
cc_test= np.load("/Users/tingchen/Desktop/Yelp Project/data_n_20_removed_all_eths_less_than_15000/y_test.npy")
locs_test = []
with open("/Users/tingchen/Desktop/Yelp Project/data_n_20_removed_all_eths_less_than_15000/locs_test.json", 'r') as file:
    locs_test = json.load(file)
    
total_X = sparse.vstack([X_val, X_test])
total_c = np.concatenate((cc_val, cc_test))
total_locs = locs_val + locs_test

ix = total_X.getnnz(1)>0
total_X = total_X[ix]
total_c= total_c[ix]
total_locs= np.array(total_locs)[ix]

### Set up inference for ccLDA

In [9]:
total_X = total_X.astype(np.int32)
total_c = total_c.astype(np.int32)

D, W = total_X.shape  # documents and vocab size
N = int(total_X.sum()) # number of total tokens
C = len(set(total_c)) # number of collections
n_topics = 25
#n_regions = self.n_regions
n_iter = 1000

# for background distribution (normal LDA)
ndz_ = np.zeros((D, n_topics), dtype=np.intc)


WS, DS = utils.matrix_to_lists(total_X) # word and document indices
CS = total_c # ethnic cuisine for each review

ZS = np.empty_like(WS, dtype=np.intc) # topics for each word

# TO DO: check if initializing xs as zeros or random is better
XS = np.random.binomial(np.ones(WS.shape[0], dtype=np.intc), .5) # indicator for background
#self.XS = XS = np.zeros(self.WS.shape[0], dtype=np.intc) # indicator for background
XS = XS.astype('intc')


np.testing.assert_equal(N, len(WS))
np.testing.assert_equal(len(set(DS)), len(CS))


for i in range(N):
    w, d= WS[i], DS[i]

    z_new = i % n_topics
    ZS[i] = z_new

    ndz_[d, z_new] += 1

all zero column in document-term matrix found


### Do sampling for document level counts only

In [10]:
nd = np.sum(ndz_, axis=1).astype(np.intc)
rng = utils.check_random_state(None)
rands = rng.rand(1024**2 // 8) 

In [11]:
random_state = utils.check_random_state(None)

In [12]:
n_topics, vocab_size = ccLDA_nzw.shape

alpha = np.repeat(0.1, n_topics).astype(np.float64)
beta = np.repeat(0.01, vocab_size).astype(np.float64)
delta = np.repeat(0.01, vocab_size).astype(np.float64) # for cross collection

In [13]:
with tqdm(total= 1000) as pbar:
    for i in range(1000):    
        random_state.shuffle(rands)

        _compare_likelihoods._sample_topics_infer(WS, DS, ZS, CS, XS,
             ccLDA_nx, ccLDA_nzw, ndz_, ccLDA_nz, ccLDA_nzwc, ccLDA_nzc, alpha,
             beta, delta, 1.0, 1.0, rands)
        
        if i % 10 ==0:
            #print("\rit: " +str(i), end="")
            #likelihoods.append(_compare_likelihoods._loglikelihood(ndz_, nd, 0.1))
            pbar.update(10)

 62%|██████▏   | 620/1000 [2:59:43<1:50:09, 17.39s/it]


KeyboardInterrupt: 

In [18]:
np.save("ccLDA_eval_ndz",ndz_)

### do c3rLDA

In [21]:
htm_ndz = np.load("./model_persistence/all_eths_less_than_15000_1000_its_25_topics/ndz.npy")
htm_nzw = np.load("./model_persistence/all_eths_less_than_15000_1000_its_25_topics/nzw.npy")

In [22]:
D, W = total_X.shape  # documents and vocab size
n_topics = 25
#n_regions = self.n_regions
n_iter = 1000
N = int(total_X.sum()) # number of total tokens

# for background distribution (normal LDA)
ndz_ = np.zeros((D, n_topics), dtype=np.intc)


WS, DS = utils.matrix_to_lists(total_X) # word and document indices

ZS = np.empty_like(WS, dtype=np.intc) # topics for each word



np.testing.assert_equal(N, len(WS))

for i in range(N):
    w, d= WS[i], DS[i]

    z_new = i % n_topics
    ZS[i] = z_new

    ndz_[d, z_new] += 1

all zero column in document-term matrix found


In [23]:
nd = np.sum(ndz_, axis=1).astype(np.intc)
nz = np.sum(htm_ndz, axis=0).astype(np.intc)

rng = utils.check_random_state(None)
rands = rng.rand(1024**2 // 8) 
random_state = utils.check_random_state(None)


n_topics, vocab_size = htm_nzw.shape

alpha = np.repeat(0.1, n_topics).astype(np.float64)
beta = np.repeat(0.01, vocab_size).astype(np.float64)

In [24]:
with tqdm(total= 500) as pbar:
    for i in range(500):    
        random_state.shuffle(rands)

        _compare_likelihoods._sample_topics_infer_base(WS, DS, ZS, htm_nzw, ndz_, nz, alpha,
             beta, rands)
        
        if i % 10 ==0:
            #print("\rit: " +str(i), end="")
            pbar.update(10)

100%|██████████| 500/500 [52:19<00:00,  6.28s/it]


In [25]:
np.save("c3rLDA_eval_ndz",ndz_)